In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import XLNetTokenizer
torch.manual_seed(1337)

In [7]:
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
n_embd = 384
n_head = 32
n_layer = 32
dropout = 0.3

train_val_ratio = 0.9
learning_rate = 3e-4

epochs = 30
num_checkpoints = 4

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
with open('../data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# -----------------------------------------
# Text tokenization
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

flatten = lambda l : [item for sublist in l for item in sublist]

def split_newline(text):
    text = text.split(' ')
    
    for i, substring in enumerate(text):
        # detect substring containing the newline symbol '\n'
        if '\n' in substring:
            # split the substring by characters
            chars = [c for c in substring]
            
            sindex = 0
            split_substring_list = []
            for j, char in enumerate(chars):
                # When meets a newline, append all previous chars as a substring ([sindex:j])
                if char == '\n':
                    c_chars = ''.join(chars[sindex:j])
                    if  c_chars != '':
                        split_substring_list.append(c_chars)
                    # Then added the '\n'
                    split_substring_list.append('\n')
                    # Update the sindex
                    sindex = j + 1
            # Added the rest of all chars as a substring
            split_substring_list.append(''.join(chars[sindex:]))
            # Replace the original string by a list of string
            text[i] = split_substring_list
        else:
            # warp other items as a list
            text[i] = [substring]
    
    # Flatten the 
    return flatten(text)

def tokenization(text):
    for i, token in enumerate(text):
        # print(token)
        if token == '\n':
            continue
        text[i] = tokenizer.tokenize(token)
    
    return flatten(text)

# here are all the unique words that occur in this text
processed_text = (tokenization(split_newline(text)))
words = sorted(list(set(processed_text)))
vocab_size = len(words)

# create a mapping from words to integers
stoi = { w:i for i, w in enumerate(words) }
itos = { i:w for i, w in enumerate(words) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a list of words, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]).replace('▁', ' ') # decoder: take a list of integers, output a words list

In [3]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [9]:
model = GPTLanguageModel()
model.load_state_dict(torch.load('../params/transformer/word_transformer64M-2023-07-16.pth'))
model.to(device)

GPTLanguageModel(
  (token_embedding_table): Embedding(9669, 384)
  (position_embedding_table): Embedding(128, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-31): 32 x Head(
            (key): Linear(in_features=384, out_features=12, bias=False)
            (query): Linear(in_features=384, out_features=12, bias=False)
            (value): Linear(in_features=384, out_features=12, bias=False)
            (dropout): Dropout(p=0.3, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.3, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_aff

In [11]:
start_token_string = 'All'
start_token = torch.tensor(stoi[start_token_string], dtype=torch.long, device=device).view(-1, 1)
print(decode(model.generate(start_token, max_new_tokens=100)[0].tolist()))

Allry,
 Than runners-or: hast distance
 For Hob, gladly all, I think him,
 As well a month of suchdeck her than the dialect than myself
 Are come into
 To search with myself of house.

 First GenARDLY:
 What thought he is a goods?

 First Gentleman:
 Thanks, in her near, but there.

 First Huntsleman:
 My gracious year:
 Go his countier, so
